In [2]:
!pip install openai-whisper
!pip install ffmpeg-python
!pip install pytube
!pip install deep-translator
!pip install silero
!pip install pydub
!pip install numpy requests
!ffmpeg -version


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=0fa65b75e816384ab44ef5a125cad9cd290b809689d7cf097689659eb9539a32
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disa

In [ ]:
# !pip install openai-whisper ffmpeg-python pytube deep-translator silero pydub numpy requests


In [4]:
import whisper
import subprocess
import os
import re

# путь к твоему видео
video_path = r"What is C Pt 1 C for Beginners.mp4"
audio_path = "temp_audio.wav"
output_text_file = "output_for_tts.txt"

# --- 1. Извлекаем аудио из видео ---
subprocess.run([
    "ffmpeg", "-i", video_path,
    "-vn",                 # без видео
    "-acodec", "pcm_s16le",
    "-ar", "16000",
    "-ac", "1",
    audio_path,
    "-y"
])

# --- 2. Загружаем модель Whisper ---
model = whisper.load_model("medium")

# --- 3. Распознаем и переводим ---
result = model.transcribe(
    audio_path,
    task="translate",      # переводим в русский текст
    language="en"          # язык исходного видео
)

raw_text = result["text"]

# --- 4. Очищаем текст — убираем мусор ---
clean_text = raw_text.strip()
clean_text = re.sub(r"\s+", " ", clean_text)  # убираем лишние пробелы

# --- 5. Сохраняем в TXT ---
with open(output_text_file, "w", encoding="utf-8") as f:
    f.write(clean_text)

print("\nФайл успешно сохранён:")
print(output_text_file)

# удалить временный аудиофайл (по желанию)
os.remove(audio_path)


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Файл успешно сохранён:
output_for_tts.txt


In [5]:
!pip install deep-translator


In [6]:
from deep_translator import GoogleTranslator

# === файлы ===
input_file = "output_for_tts.txt"
output_file = "output_ru.txt"

# === читаем текст ===
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().strip()

# === функция для разбиения текста ===
def split_text(text, max_len=4000):
    parts = []
    while len(text) > max_len:
        # ищем ближайшую точку, чтобы не рвать предложения
        cut = text.rfind(".", 0, max_len)
        if cut == -1:
            cut = max_len
        parts.append(text[:cut+1].strip())
        text = text[cut+1:].strip()
    parts.append(text)
    return parts

chunks = split_text(text, 4000)

print(f"Найдено частей: {len(chunks)}")

translated_parts = []

# === переводим по частям ===
for i, chunk in enumerate(chunks, 1):
    print(f"Перевод части {i}/{len(chunks)}...")
    translated = GoogleTranslator(source="auto", target="ru").translate(chunk)
    translated_parts.append(translated)

# === сохраняем в файл ===
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n\n".join(translated_parts))

print("\nГотово! Переведённый файл сохранён как:", output_file)


Найдено частей: 1
Перевод части 1/1...

Готово! Переведённый файл сохранён как: output_ru.txt


In [7]:
#@title Install dependencies

!pip install -q omegaconf

import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

In [8]:
# see latest avaiable models
available_languages = list(models.tts_models.keys())
print(f'Available languages {available_languages}')

for lang in available_languages:
    _models = list(models.tts_models.get(lang).keys())
    print(f'Available models for {lang}: {_models}')

Available languages ['ru', 'en', 'de', 'es', 'fr', 'ba', 'xal', 'tt', 'uz', 'ua', 'indic', 'cyrillic', 'multi']
Available models for ru: ['v5_cis_base', 'v5_cis_base_nostress', 'v5_cis_ext', 'v5_1_ru', 'v5_ru', 'v4_ru', 'v3_1_ru', 'ru_v3', 'aidar_v2', 'aidar_8khz', 'aidar_16khz', 'baya_v2', 'baya_8khz', 'baya_16khz', 'irina_v2', 'irina_8khz', 'irina_16khz', 'kseniya_v2', 'kseniya_8khz', 'kseniya_16khz', 'natasha_v2', 'natasha_8khz', 'natasha_16khz', 'ruslan_v2', 'ruslan_8khz', 'ruslan_16khz']
Available models for en: ['v3_en', 'v3_en_indic', 'lj_v2', 'lj_8khz', 'lj_16khz']
Available models for de: ['v3_de', 'thorsten_v2', 'thorsten_8khz', 'thorsten_16khz']
Available models for es: ['v3_es', 'tux_v2', 'tux_8khz', 'tux_16khz']
Available models for fr: ['v3_fr', 'gilles_v2', 'gilles_8khz', 'gilles_16khz']
Available models for ba: ['aigul_v2']
Available models for xal: ['v3_xal', 'erdni_v2']
Available models for tt: ['v3_tt', 'dilyara_v2']
Available models for uz: ['v4_uz', 'v3_uz', 'dilna

In [9]:
import torch

language = 'ru'
model_id = 'v5_ru'
device = torch.device('cpu')

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

/usr/local/lib/python3.12/dist-packages/torch/hub.py:335: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/snakers4/silero-models/zipball/master" to /root/.cache/torch/hub/master.zip


100%|██████████| 139M/139M [00:08<00:00, 17.6MB/s]
/usr/local/lib/python3.12/dist-packages/torch/package/package_importer.py:257: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  dtype = storage_type.dtype


In [10]:
model.speakers

['aidar', 'baya', 'kseniya', 'eugene', 'xenia']

In [11]:
from pydub import AudioSegment
import re
import numpy as np

sample_rate = 48000
speaker = 'xenia'
put_accent=True
put_yo=True
put_stress_homo=True
put_yo_homo=True

# читаем текст из файла
with open("output_ru.txt", "r", encoding="utf-8") as f:
    example_text = f.read().strip()

# разбиение текста на куски
def split_text_safe(text, max_len=700):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ""
    for s in sentences:
        if len(current) + len(s) + 1 > max_len:
            chunks.append(current.strip())
            current = s
        else:
            current += " " + s
    if current:
        chunks.append(current.strip())
    return chunks

chunks = split_text_safe(example_text, 700)
print(f"Количество частей: {len(chunks)}")

final_audio = AudioSegment.empty()
silence = AudioSegment.silent(duration=400)  # 400 мс пауза

for i, chunk in enumerate(chunks, 1):
    print(f"Озвучиваем часть {i}/{len(chunks)}...")
    audio_chunk = model.apply_tts(
        text=chunk,
        speaker=speaker,
        sample_rate=sample_rate,
        put_accent=put_accent,
        put_yo=put_yo,
        put_stress_homo=put_stress_homo,
        put_yo_homo=put_yo_homo
    )
    audio_np = audio_chunk.cpu().numpy()
    audio_np_int16 = (audio_np * 32767).astype(np.int16)  # улучшение качества
    seg = AudioSegment(
        audio_np_int16.tobytes(),
        frame_rate=sample_rate,
        sample_width=2,
        channels=1
    )
    final_audio += seg + silence

final_audio.export("output_final_clean.wav", format="wav")
print("Готово! Итоговый файл с улучшенным качеством: output_final_clean.wav")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Количество частей: 3
Озвучиваем часть 1/3...
Озвучиваем часть 2/3...
Озвучиваем часть 3/3...
Готово! Итоговый файл с улучшенным качеством: output_final_clean.wav


In [12]:
import subprocess
from moviepy.editor import VideoFileClip, ColorClip, concatenate_videoclips

video_path = r"What is C Pt 1 C for Beginners.mp4"
audio_path = "output_final_clean.wav"
output_path = "final_video.mp4"


clip = VideoFileClip(video_path).subclip(10)  # пропускаем первые 10 секунд
clip.write_videofile(output_path, codec="libx264")
video_clip = clip
video_duration = video_clip.duration

# 2️⃣ Получаем длительность аудио
import wave

with wave.open(audio_path, "r") as wav_file:
    frames = wav_file.getnframes()
    rate = wav_file.getframerate()
    audio_duration = frames / float(rate)

print(f"Видео длительность: {video_duration}s, Аудио длительность: {audio_duration}s")

# 3️⃣ Добавляем черный экран, если аудио длиннее видео
if audio_duration > video_duration:
    extra_duration = audio_duration - video_duration
    black_clip = ColorClip(size=video_clip.size, color=(0,0,0), duration=extra_duration)
    video_clip = concatenate_videoclips([video_clip, black_clip])

# 4️⃣ Сохраняем видео без звука во временный файл
temp_video = "temp_video_no_audio.mp4"
video_clip.write_videofile(temp_video, audio=False, codec="libx264", fps=video_clip.fps)

# 5️⃣ Используем ffmpeg для замены аудио
subprocess.run([
    "ffmpeg",
    "-y",  # перезаписать если есть
    "-i", temp_video,
    "-i", audio_path,
    "-c:v", "copy",
    "-c:a", "aac",
    "-b:a", "192k",
    "-shortest",  # чтобы видео длилось по меньшей длительности
    output_path
])

print("Готово! Итоговое видео:", output_path)


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



t: 100%|█████████▉| 2783/2784 [04:10<00:00, 11.19it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file What is C Pt 1 C for Beginners.mp4, 6220800 bytes wanted but 0 bytes read,at frame 3082/3084, at time 102.84/102.89 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready final_video.mp4
Видео длительность: 92.89s, Аудио длительность: 102.29975s
Moviepy - Building video temp_video_no_audio.mp4.
Moviepy - Writing video temp_video_no_audio.mp4



Moviepy - Done !
Moviepy - video ready temp_video_no_audio.mp4
Готово! Итоговое видео: final_video.mp4
